## API Data Wrangling Mini Project
        By Muluemebet Ayalew
           March, 2020


This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY= "********************"  # API key hidden for security reason 

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests                            # to get data from url
from  collections import defaultdict       # to collect data in dictionary form


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url= "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?" # link for request
res= requests.get(url, params=dict(key=API_KEY)) # get the data using api _key



The json is structured as key-value pair. The key "dataset_data" contains all relevant informations such as column names, start and end date of the collected data, frequency, and  data as dictionary. The "data" key then contains list of daily records on which we are interested for analysis.

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json=res.json() # get data in json format
json;

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1) Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
data= json["dataset_data"]["data"]  # get the actual record from the json file
data_2017= [day for day in data if day[0][:4]=='2017'] #  collect data only for the year 2017
#print(data_2017)

### 2) Convert the returned JSON object into a Python dictionary
I assum that the followup questions are based on the year 2017. Therefore, this question is addresed based on the extracted data for the year 2017. The approach is the same if we are interested to collect the whole dataset.

In [6]:
columns= json['dataset_data']['column_names'] # Extract the column names 
print(columns)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [7]:
dic= defaultdict(list)  # a dictionary its keys are column names and its values are list
for day in data_2017:
    for col, value in zip(columns, day):
        dic[col].append(value) 

In [8]:
dic.keys() # columns

dict_keys(['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'])

In [9]:
dic ; # Answer 

### 3) Calculate what the highest and lowest opening prices were for the stock in this period.

In [10]:
# collect the price if it is not None and save to cleaned_open
cleaned_open= [price for price in dic["Open"] if price] 

In [11]:
print("Original length: {}". format(len(dic["Open"])))        # the length is 255 with None values
print("Length after cleaned: {}". format(len(cleaned_open)))  # Now the length is 252 after droping the None values 

Original length: 255
Length after cleaned: 252


In [12]:
max(cleaned_open)  # Answer, Highest opening price in 2017 

53.11

In [13]:
min(cleaned_open)  # Answer, lowest opening price in 2017

34.0

### 4) What was the largest change in any one day (based on High and Low price)?

In [14]:
# place holder to collect the difference between the highest and lowest price in a day
change_high_low= [] 

for i in range(len(dic["High"])):
    diff= dic["High"][i]- dic["Low"][i]  # difference between highest and lowest price
    change_high_low.append(diff)

In [15]:
max(change_high_low)  # Answer, the largest change(high - low) in price in a day (2017)

2.8100000000000023

### 5) What was the largest change between any two days (based on Closing Price)?

Since the data is ordered in descending order of date, that means the most recent dates are at the top, we need to reverse the data to compute the change in closing price between two day. So that the days in January 2017 will be at the top and December at the bottom. 


In [16]:
dic["Date"][:6] # look how the date is sorted . It was sorted in descending order 

['2017-12-29',
 '2017-12-28',
 '2017-12-27',
 '2017-12-22',
 '2017-12-21',
 '2017-12-20']

In [17]:
dic["Date"][-6:] # the last six days 

['2017-01-09',
 '2017-01-06',
 '2017-01-05',
 '2017-01-04',
 '2017-01-03',
 '2017-01-02']

In [18]:
# the closing price from January 2017 to December 2017, obtained after reversing the original list
reversed_close= dic["Close"][::-1] 

In [19]:
# compute the change in closing price  between two consequative working days
change_close=[dic["Close"][i+1]- dic["Close"][i] for i in range((len(dic["Close"])-1))]
    

In [20]:
max(change_close) # Answer, the largest change in closing price between two consequetive working days

2.559999999999995

### 6) What was the average daily trading volume during this year?

In [21]:
dic.keys()

dict_keys(['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'])

In [22]:
total_volume=sum(dic['Traded Volume'])  # sum of traded volume
length=len(dic['Traded Volume'])        # number of days stock exchanged in 2017 

In [23]:
average= total_volume/length     # average daily trading volume in 2017
print(average)                   # answer 

89124.33725490196


### 7) (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [24]:
def median(lst):
    """compute the median of elements in the list, lst"""
    
    lst=sorted(lst) # sort the list in ascending order 
    length =len(lst)  # the length of the list
    
    if length %2==0:         # if length is even
        index= length//2     # make integer division
        
        # the average of the the two middles. Since python indexing starts with zero,one is deducted from index 
        median= (lst[index-1] +lst[(index)])/2  # the average of the the two middles, since python indexing starts with zero
   
    else:  #length is odd
        index=(length +1)//2
        median=lst[index-1]  # adjusted to zero indexing of python
   
    return median   

In [25]:
median(dic['Traded Volume']) # Answer, median of trading volume in 2017

76286.0